Image Segmentation use: https://docs.ultralytics.com/modes/predict/#inference-sources

For model Training use: https://github.com/computervisioneng/image-classification-yolov8/tree/main/local_env

Hyper parameter tuning: https://docs.ultralytics.com/guides/hyperparameter-tuning/


## Importing Libraries

Importing the necessary libraries for the project

And initializing constants

```python

```


In [171]:
%pip install opencv-python-headless numpy ultralytics Pillow

Note: you may need to restart the kernel to use updated packages.


In [219]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps

In [240]:
### CONSTANTS
# Image properties
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
IMAGE_CHANNELS = 3

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif')


# Director paths
PROCESSED_DS_PATH_NM = './processed_dataset/NoMask'
PROCESSED_DS_PATH_M = './processed_dataset/Mask'
RAW_DS_PATH = './Data'

## MODELS
# Segmentation
SEG_KWARGS = {
    'classes': 0,
    'imgsz': IMAGE_SIZE,
    'conf': 0.5,
    'boxes':True,
    'save':True,
    'show_labels':False,
    'show_conf':True,
    'max_det':1,
}
# Classification
CLASS_KWARGS = {
    'epochs': 20,
    'project': './models',
}
MODEL_KWARGS = {
    'save': True,
    'project': './',
    'name': 'tests',
}

## Preparig data for training


### Preparing Models

```python

```


In [221]:
img_seg_model = YOLO('yolov8n-seg.pt')
img_class_model = YOLO("yolov8n-cls.pt")

In [217]:
with Image.open('processed_dataset\\Mask\\val\\6\\sthephany_6_0.jpg') as img:

    # add a border
    img = ImageOps.expand(img,border=50,fill='black')

    # Enhancing the sharpness
    enhancer = ImageEnhance.Sharpness(img)
    enhanced_img = enhancer.enhance(7)
    
    
    # Save it to a new file
    enhanced_img.save('./result.jpg')

In [ ]:
img_seg_model('./result.jpg', **SEG_KWARGS)

### Preparing dataset

```python

```


In [231]:
def process_images(
        source_folder=RAW_DS_PATH, 
        destination_folder=PROCESSED_DS_PATH_NM, 
        grayscale=True, 
        resize=True,
        resize_dimensions=IMAGE_SIZE):
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    os.makedirs(destination_folder, exist_ok=True)

    for subdir, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(IMAGE_EXT):
                source_path = os.path.join(subdir, file)
                relative_path = os.path.relpath(subdir, source_folder)
                destination_subdir = os.path.join(destination_folder, relative_path)
                os.makedirs(destination_subdir, exist_ok=True)

                destination_path = os.path.join(destination_subdir, file)

                image = cv2.imread(source_path)
                if image is None:
                    print(f"Unable to read image: {source_path}")
                    continue

                if grayscale:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                if resize:
                    image = cv2.resize(image, resize_dimensions)

                cv2.imwrite(destination_path, image)

In [229]:
def mask_images(
        source_folder = PROCESSED_DS_PATH_NM,
        destination_folder=PROCESSED_DS_PATH_M):
    
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return
    # Calculate the length of the root_path plus one for the trailing path separator
    root_path_length = len(source_folder) + len(os.path.sep)

    # Set comprehension to get only directories at depth 2
    subdirs = {subdir[root_path_length:] for subdir, dirs, files in os.walk(source_folder)
               if subdir.count(os.path.sep) - source_folder.count(os.path.sep) == 2}
    
    
    for subdir in subdirs:
        img_folder = os.path.join(source_folder, subdir)
        img_seg_model(img_folder, project=destination_folder, name=subdir, **SEG_KWARGS)
    

In [237]:
process_images()

In [235]:
SEG_KWARGS

{'classes': 0,
 'imgsz': (356, 356),
 'conf': 0.5,
 'boxes': True,
 'save': True,
 'show_labels': False,
 'show_conf': True,
 'max_det': 1}

In [241]:
mask_images()


image 1/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\train\5\img_100379_jpg.rf.7bd1fbcc69c658788a477e8a28cd13cc.jpg: 384x384 1 person, 55.0ms
image 2/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\train\5\img_100431_jpg.rf.205c629ec12dde48c8b7d583ac88dfa7.jpg: 384x384 (no detections), 49.0ms
image 3/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\train\5\img_100431_jpg.rf.84a6f70730b06cb38e24f41af48fead1.jpg: 384x384 1 person, 47.0ms
image 4/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\train\5\img_100589_jpg.rf.7064fb755d873b81f54278e59fc341fa.jpg: 384x384 (no detections), 45.0ms
image 5/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\train\5\img_100589_jpg.rf.d55c2bf75792bb7e596d96bc5777d086.jpg: 384x384 (no detections), 47.0ms
image 6/167 c:\Users\Magnico\Documents\GitProjects\driver_detection\

## Training

```python

```


In [11]:
try:
    run = max(os.listdir(CLASS_KWARGS['project']))
    run = 1 + int(run)
except:
    run = 1

img_class_model.train(data=PROCESSED_DS_PATH_NM, name=run, **CLASS_KWARGS)

Ultralytics YOLOv8.0.207  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=./processed_dataset/NoMask, epochs=20, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=./models, name=3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002852A396DC0>
curves: []
curves_results: []
fitness: 0.6233480274677277
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.27753305435180664, 'metrics/accuracy_top5': 0.9691630005836487, 'fitness': 0.6233480274677277}
save_dir: WindowsPath('models/3')
speed: {'preprocess': 0.0, 'inference': 4.271863315599081, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.27753305435180664
top5: 0.9691630005836487

## Performance Evaluation

In [ ]:
cm_path = os.path.join(CLASS_KWARGS['project'], str(run), 'confusion_matrix.jpg')
results_path = os.path.join(CLASS_KWARGS['project'], str(run), 'results.jpg')

cm_img = plt.imread(cm_path)
results_img = plt.imread(results_path)

plt.imshow(cm_img)
plt.show()

plt.imshow(results_img)
plt.show()

## Prediction


In [13]:
def get_model(run, best=False):
    weight = 'weights/best.pt' if best else 'weights/last.pt'
    path = os.path.join(CLASS_KWARGS['project'], str(run), weight)
    return YOLO(path)

In [14]:
def predict(image_path):
    try:
        runs = os.listdir(CLASS_KWARGS['project'])
    except:
        print("No runs found")
        return
    i = input(f"Choose a run from {runs}: ")

    while i not in runs:
        i = input(f"Choose a run from {runs}: ")
    
    model = get_model(i)

    model(image_path, **MODEL_KWARGS)

In [15]:
predict('processed_dataset\\NoMask\\val\\0\\dilan_145.jpg')


image 1/1 c:\Users\Magnico\Documents\GitProjects\driver_detection\processed_dataset\NoMask\val\0\dilan_145.jpg: 224x224 6 0.22, 2 0.19, 5 0.17, 4 0.15, 3 0.12, 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Results saved to tests
